## ECI 249 Term Paper

Kenneth Larrieu

Many factors must be considered for salmon habitat restoration in rivers. Software is currently being developed for assessing different restoration options. The RiverArchitect software described in Schwindt et al. (2019) estimates the cost of different restoration projects, the spatial distribution of stage-dependent salmon habitat suitability throughout the modeled area, as well as the expected lifespan of the habitat areas (as they are subject to change via morphodynamic processes). A common restoration procedure in rivers which are deprived of sediment (e.g. due to damming) is gravel augmentation. The introduction of gravel into the river channel may enhance the suitability of salmon habitat, however, once the sediment is transported downstream, the habitat suitability of the river reach may decrease again. Using larger gravel will increase the lifespan of the restoration features, but if the gravel is too large it will also be unsuitable for salmon habitat. Therefore, one might want to optimize a gravel distribution throughout a river reach of interest, maximizing the habitat suitability and restoration feature lifespan.

The habitat suitability can be quantified by a habitat suitability index (HSI), which ranges from 0 to 1, with 1 indicating ideal habitat. River Architect includes a module for calculating spatially explicit habitat suitability throughout a river reach.

The lifespan of a restoration feature can be estimated as the recurrence interval of the corresponding flood at which the sediment becomes mobilized. Sediment is mobilized when the Shields stress (dimensionless shear stress) exceeds a critical value.

Using hydrodynamic model results, River Architect can then calculate the spatially distributed maximum moblie sediment size. This is then used as a potential distribution $D$ to be used for gravel augmentation, which would need to be replenished in the recurrence interval of the corresponding flood on average.

One of the parameters used that may have uncertainty is the critical Shields stress for sediment mobilization $(\tau^*_{cr})$.

In [ ]:
A